In [ ]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5
%additional_python_modules awswrangler, great_expectations, https://github.com/julioasotodv/spark-df-profiling/archive/master.zip, pyyaml, datacompy


import sys
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, isnull, trim, avg, count, sum
from pyspark.sql.types import StringType, IntegerType
import awswrangler as wr
import spark_df_profiling
from pyspark.context import SparkContext
from awsglue.context import GlueContext
import great_expectations as ge
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import DataContextConfig, S3StoreBackendDefaults
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.core.batch import RuntimeBatchRequest
import boto3
import yaml
import re
import sys
from awsglue.utils import getResolvedOptions
from great_expectations.data_context.types.resource_identifiers import ValidationResultIdentifier, \
    ExpectationSuiteIdentifier, BatchIdentifier
s3 = boto3.resource("s3")
bucket = s3.Bucket('atividade5-raw')
s3_client = boto3.client('s3')
response = s3_client.get_object(Bucket='atividade5-raw',
                                Key="great_expectations/great_expectations.yaml")
configfile = yaml.safe_load(response["Body"])
folder = "great_expectations/"
bucketNameRaw = "atividade5-raw/"
bucketNameTrusted = "atividade5-trusted/"

pastaBancos = "bancos/"
pastaEmpregados = "empregados/"
pastaReclamacoes = "reclamacoes/"
df_bancos = spark.read.option("delimiter", "\t").option("header", True).option("charset", "latin1").csv('s3://'+bucketNameRaw+pastaBancos)
df_empregados = spark.read.option("delimiter", "|").option("header", True).csv('s3://'+bucketNameRaw+pastaEmpregados)
df_reclamacoes = spark.read.option("delimiter", ";").option("header", True).csv('s3://'+bucketNameRaw+pastaReclamacoes)
profile_bancos = spark_df_profiling.ProfileReport(df_bancos, title="pyspark_bancos", minimal=True)
bucket.put_object(Key=folder + 'pyspark_bancos.html', Body=profile_bancos.rendered_html(), ContentType='text/html')
config = DataContextConfig(config_version=configfile['config_version'], datasources=configfile['datasources'],
                           expectations_store_name=configfile['expectations_store_name'],
                           validations_store_name=configfile['validations_store_name'],
                           evaluation_parameter_store_name=configfile['evaluation_parameter_store_name'],
                           plugins_directory='/great_expectations/plugins',
                           stores=configfile['stores'],
                           data_docs_sites=configfile['data_docs_sites'],
                           store_backend_defaults=S3StoreBackendDefaults(
                               default_bucket_name=configfile['data_docs_sites']['s3_site']['store_backend']['bucket']))
profile_empregados = spark_df_profiling.ProfileReport(df_empregados, title="pyspark_empregados", minimal=True)
bucket.put_object(Key=folder + 'pyspark_empregados.html', Body=profile_empregados.rendered_html(), ContentType='text/html')
config = DataContextConfig(config_version=configfile['config_version'], datasources=configfile['datasources'],
                           expectations_store_name=configfile['expectations_store_name'],
                           validations_store_name=configfile['validations_store_name'],
                           evaluation_parameter_store_name=configfile['evaluation_parameter_store_name'],
                           plugins_directory='/great_expectations/plugins',
                           stores=configfile['stores'],
                           data_docs_sites=configfile['data_docs_sites'],
                           store_backend_defaults=S3StoreBackendDefaults(
                               default_bucket_name=configfile['data_docs_sites']['s3_site']['store_backend']['bucket']))
profile_reclamacoes = spark_df_profiling.ProfileReport(df_reclamacoes, title="pyspark_reclamacoes", minimal=True)
bucket.put_object(Key=folder + 'pyspark_reclamacoes.html', Body=profile_reclamacoes.rendered_html(), ContentType='text/html')
config = DataContextConfig(config_version=configfile['config_version'], datasources=configfile['datasources'],
                           expectations_store_name=configfile['expectations_store_name'],
                           validations_store_name=configfile['validations_store_name'],
                           evaluation_parameter_store_name=configfile['evaluation_parameter_store_name'],
                           plugins_directory='/great_expectations/plugins',
                           stores=configfile['stores'],
                           data_docs_sites=configfile['data_docs_sites'],
                           store_backend_defaults=S3StoreBackendDefaults(
                               default_bucket_name=configfile['data_docs_sites']['s3_site']['store_backend']['bucket']))
column_to_rename_old = ['Segmento','CNPJ','Nome']
column_to_rename_new = ['segmento','cnpj','nome']
for i in range(len(column_to_rename_old)):
    df_bancos = df_bancos.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
df_bancos = df_bancos.withColumn("nome", regexp_replace("nome", " - PRUDENCIAL", ""))
column_to_rename_old = ['employer_name', 'reviews_count', 'culture_count', 'salaries_count', 'benefits_count', 'employer-website', 'employer-headquarters', 'employer-founded', 'employer-industry', 'employer-revenue', 'url', 'Geral', 'Cultura e valores', 'Diversidade e inclusao', 'Qualidade de vida', 'Alta lideranca', 'Remuneracao e beneficios', 'Oportunidades de carreira', 'Recomendam para outras pessoas(%)', 'Perspectiva positiva da empresa(%)', 'CNPJ', 'Nome', 'match_percent']
column_to_rename_new = ['employer_name', 'reviews_count', 'culture_count', 'salaries_count', 'benefits_count', 'employer_website', 'employer_headquarters', 'employer_founded', 'employer_industry', 'employer_revenue', 'url', 'geral', 'cultura_e_valores', 'diversidade_e_inclusao', 'qualidade_de_vida', 'alta_lideranca', 'remuneracao_e_beneficios', 'oportunidades_de_carreira', 'recomendam_para_outras_pessoas_perc', 'perspectiva_positiva_da_empresa_perc', 'cnpj_emp', 'nome_emp', 'match_percent']

for i in range(len(column_to_rename_old)):
    df_empregados = df_empregados.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])

columns = ["nome_emp", col("geral").cast("float").alias("geral"),col("remuneracao_e_beneficios").cast("float").alias("remuneracao_e_beneficios")]

df_empregados = df_empregados.select(columns)
column_to_rename_old = ['Ano', 'Trimestre', 'Categoria', 'Tipo', 'CNPJ IF', 'Instituicao financeira', 'indice', 'Quantidade de reclamacoes reguladas procedentes', 'Quantidade de reclamacoes reguladas - outras', 'Quantidade de reclamacoes nno reguladas', 'Quantidade total de reclamacoes', 'Quantidade total de clientes - CCS e SCR', 'Quantidade de clientes - CCS', 'Quantidade de clientes - SCR',',,']
column_to_rename_new = ['ano', 'trimestre', 'categoria', 'tipo', 'cnpj_if', 'instituicao_financeira', 'indice', 'quantidade_de_reclamacoes_reguladas_procedentes', 'quantidade_de_reclamacoes_reguladas_outras', 'quantidade_de_reclamacoes_nao_reguladas', 'quantidade_total_de_reclamacoes', 'quantidade_total_de_clientes_ccs_e_scr', 'quantidade_de_clientes_ccs', 'quantidade_de_clientes_scr', ',,']
for i in range(len(column_to_rename_old)):
    df_reclamacoes = df_reclamacoes.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
df_reclamacoes = df_reclamacoes.withColumn("trimestre", regexp_replace("trimestre", "o", ""))
df_reclamacoes = df_reclamacoes.withColumn("instituicao_financeira", regexp_replace("instituicao_financeira", "conglomerado", ""))
df_reclamacoes = df_reclamacoes.withColumn("instituicao_financeira", regexp_replace("instituicao_financeira","[()]",""))
df_reclamacoes = df_reclamacoes.withColumn("instituicao_financeira",trim(col("instituicao_financeira")))
df_reclamacoes = df_reclamacoes.withColumn("indice", regexp_replace("indice",",","."))
columns = [col("ano").cast("int").alias("ano"), "trimestre",col("instituicao_financeira").alias("nome_rec"),col("indice").cast("float").alias("indice"),\
           col("quantidade_total_de_reclamacoes").cast("int").alias("qtd_reclamacoes"),col("quantidade_total_de_clientes_ccs_e_scr").cast("int").alias("qtd_clientes")]

df_reclamacoes = df_reclamacoes.select(columns)
df_bancos.write.mode("overwrite").parquet('s3://'+bucketNameTrusted+pastaBancos)
df_empregados.write.mode("overwrite").parquet('s3://'+bucketNameTrusted+pastaEmpregados)
df_reclamacoes.write.mode("overwrite").parquet('s3://'+bucketNameTrusted+pastaReclamacoes)
job.commit()